# Import packages

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

# Import  and merge data from oecd-files (Yueqi & Arabella)

In [2]:
# Import OECD-data from CSV-files
oecd_data1 = pd.read_csv('Data/CITIES_POPULATION.csv', sep="|", header=0)
oecd_data2 = pd.read_csv('Data/CITIES_AGE.csv', sep="|", header=0)
oecd_data3 = pd.read_csv('Data/CITIES_ECONOMY.csv', sep="|", header=0)
oecd_data4 = pd.read_csv('Data/CITIES_LABOUR.csv', sep="|", header=0)
oecd_data5 = pd.read_csv('Data/CITIES_TERRITORY.csv', sep="|", header=0)
oecd_data6 = pd.read_csv('DATA/CITIES_ENVIRONMENT.csv', sep="|", header=0)

In [3]:
# Put all OECD-data into dataframe and append all dataframes
oecd_data_df = [oecd_data1, oecd_data2, oecd_data3, oecd_data4, oecd_data5, oecd_data6]
# Call concat method
oecd_df = pd.concat(oecd_data_df)

In [4]:
# first, we print the oecd file of cities_territory, check what variables we have
print(oecd_data5)

      METRO_ID Metropolitan areas            VAR  \
0        AT001             Vienna           SURF   
1        AT001             Vienna       URB_AREA   
2        AT001             Vienna       URB_AREA   
3        AT002               Graz           SURF   
4        AT002               Graz       URB_AREA   
...        ...                ...            ...   
56216    MEX64             Carmen       URB_AREA   
56217    MEX64             Carmen  URB_AREA_CORE   
56218    MEX64             Carmen  URB_AREA_CORE   
56219    MEX64             Carmen    URB_AREA_PC   
56220    MEX64             Carmen    URB_AREA_PC   

                                               Variables  TIME  Year  \
0                      Metropolitan area total land area  2018  2018   
1      Urbanised area (built-up area or land for urba...  2000  2000   
2      Urbanised area (built-up area or land for urba...  2014  2014   
3                      Metropolitan area total land area  2018  2018   
4      Urbanise

# Tip 1: calculate construction ratio out of oecd_data5

In [5]:
#pick the Territory dataset, calculate the construction densiy ratio
oecd_data5_1 = oecd_data5.loc[oecd_data5['Variables']=='Urbanised area (built-up area or land for urban use in km2)']

oecd_data5_2 = oecd_data5.loc[oecd_data5['Variables']=='Metropolitan area total land area']


#oecd_data5_1 has 1336 rows, while oecd_data5_2 has 668 rows (exactly two time of the first dataset), 
#we could try to combine the two datasets by the common column "METRO_ID",

oecd_data5_improved = pd.merge(oecd_data5_1,oecd_data5_2,how='inner',on='METRO_ID')

#and then calculate the constuction density ratio = "Urbanised area" / "Metropolitan area total land area"
oecd_data5_improved['construction_ratio'] = oecd_data5_improved['Value_x'] / oecd_data5_improved['Value_y']
print(oecd_data5_improved)

     METRO_ID Metropolitan areas_x     VAR_x  \
0       AT001               Vienna  URB_AREA   
1       AT001               Vienna  URB_AREA   
2       AT002                 Graz  URB_AREA   
3       AT002                 Graz  URB_AREA   
4       AT003                 Linz  URB_AREA   
...       ...                  ...       ...   
1331    JPN28            Takamatsu  URB_AREA   
1332    JPN31               Numazu  URB_AREA   
1333    JPN31               Numazu  URB_AREA   
1334    MEX64               Carmen  URB_AREA   
1335    MEX64               Carmen  URB_AREA   

                                            Variables_x  TIME_x  Year_x  \
0     Urbanised area (built-up area or land for urba...    2000    2000   
1     Urbanised area (built-up area or land for urba...    2014    2014   
2     Urbanised area (built-up area or land for urba...    2000    2000   
3     Urbanised area (built-up area or land for urba...    2014    2014   
4     Urbanised area (built-up area or land for 

# Tip2 get the tree-cover ratio out of oecd_data6

In [6]:
#check the oecd_data6
print(oecd_data6)
#we already have the tree-cover ratio

       METRO_ID Metropolitan areas                   VAR  \
0         USA62            Hidalgo               SPEX_10   
1         USA62            Hidalgo               SPEX_10   
2         USA62            Hidalgo               SPEX_10   
3         USA62            Hidalgo               SPEX_10   
4         USA62            Hidalgo               SPEX_10   
...         ...                ...                   ...   
110603    USA99           Mahoning          SPEX_35_CORE   
110604    USA99           Mahoning          SPEX_35_CORE   
110605    USA99           Mahoning  TREECOVER_SHARE_CORE   
110606    USA99           Mahoning  TREECOVER_SHARE_CORE   
110607    USA99           Mahoning  TREECOVER_SHARE_CORE   

                                                Variables  TIME  Year  \
0       Share of population exposed to a level of PM2....  2000  2000   
1       Share of population exposed to a level of PM2....  2005  2005   
2       Share of population exposed to a level of PM2....  2

In [7]:
# TEMPORARY: visualize the results
oecd_df

,METRO_ID,Metropolitan areas,VAR,Variables,TIME,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,USA116,Allen,T_T,"Population, all ages, administrative data",2001,2001,PER,Persons,0,Units,NaN,NaN,365954.0,NaN,NaN
1,USA116,Allen,T_T,"Population, all ages, administrative data",2002,2002,PER,Persons,0,Units,NaN,NaN,368402.0,NaN,NaN
2,USA116,Allen,T_T,"Population, all ages, administrative data",2003,2003,PER,Persons,0,Units,NaN,NaN,371190.0,NaN,NaN
3,USA116,Allen,T_T,"Population, all ages, administrative data",2004,2004,PER,Persons,0,Units,NaN,NaN,372757.0,NaN,NaN
4,USA116,Allen,T_T,"Population, all ages, administrative data",2005,2005,PER,Persons,0,Units,NaN,NaN,375420.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110603,USA99,Mahoning,SPEX_35_CORE,Share of population in the core area exposed t...,2018,2018,NaN,NaN,0,Units,NaN,NaN,0.0,NaN,NaN
110604,USA99,Mahoning,SPEX_35_CORE,Share of population in the core area exposed t...,2019,2019,NaN,NaN,0,Units,NaN,NaN,0.0,NaN,NaN
110605,USA99,Mahoning,TREECOVER_SHARE_CORE,"Tree cover, as a share of the metropolitan cor...",2004,2004,NaN,NaN,0,Units,NaN,NaN,21.6,NaN,NaN
110606,USA99,Mahoning,TREECOVER_SHARE_CORE,"Tree cover, as a share of the metropolitan cor...",2015,2015,NaN,NaN,0,Units,NaN,NaN,20.8,NaN,NaN


## Rename columnames

In [8]:
# create a dictionary where key = old name and value = new name
dict = {'METRO_ID': 'metroId',
        'Metropolitan areas': 'metropolitanAreas',
        'VAR': 'var',
       'Variables' : 'variables',
        'TIME' : 'time',
        'Year' : 'year',
        'Unit Code': 'unitCode',
        'Unit' : 'unit',
        'PowerCode Code' : 'powerCodeCode',
        'PowerCode': 'powerCode',
        'Reference Period Code' : 'referencePeriodCode',
        'Reference Period' : 'referencePeriod',
        'Value':'value',
        'Flag Codes' : 'flagCodes',
        'Flags': 'flags'
       }
  
# call rename () method
oecd_df.rename(columns=dict,
          inplace=True)

In [9]:
# Checking the data
print(oecd_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586301 entries, 0 to 110607
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   metroId              586301 non-null  object 
 1   metropolitanAreas    586301 non-null  object 
 2   var                  586301 non-null  object 
 3   variables            586301 non-null  object 
 4   time                 586301 non-null  int64  
 5   year                 586301 non-null  int64  
 6   unitCode             138868 non-null  object 
 7   unit                 138868 non-null  object 
 8   powerCodeCode        586301 non-null  int64  
 9   powerCode            586301 non-null  object 
 10  referencePeriodCode  0 non-null       float64
 11  referencePeriod      0 non-null       float64
 12  value                584753 non-null  float64
 13  flagCodes            29450 non-null   object 
 14  flags                29450 non-null   object 
dtypes: float64(3), in

# Add longitude and latitude to join data
Strings that represent metropolitan areas are often different, which makes it difficult to join data from different sources. For example, "Lafayette (IN)" and "Lafayette, IN" and "Lafayette". Therefore, longitude and latitude are added as a unique identifier to join data.
## Create unique list of metropolitans from dataframe

In [10]:
# The length of the list is determined first, to determine which function to apply
len(oecd_df['metropolitanAreas'].unique().tolist())

665

In [11]:
# Assign the list to a variable
metro_list= oecd_df['metropolitanAreas'].unique().tolist()
# Convert the list into a dataframe
df_metropolitan=DataFrame (metro_list, columns=['metropolitanAreas'] )
# Check the dataframe
df_metropolitan

,metropolitanAreas
0,Allen
1,Warsaw
2,Kagoshima
3,Mexicali
4,Leicester
...,...
660,Cass
661,Carmen
662,Linn
663,Lafayette (IN)


## Add longitudes and latitudes to metropolitans

In [12]:
# declare an empty list to store latitude and longitude of values of the metropolitanAreas column
longitude = []
latitude = []

# use function to find the coordinate of a given metropolitanArea
def findGeocode(city):

    # try and catch is used to overcome the exception thrown by geolocator using geocodertimedout
    try:
        # Specify the user_agent as app name: this is my (Arabella) local address
        geolocator = Nominatim(user_agent="185.18.149.47") 
        return geolocator.geocode(city)
    except GeocoderTimedOut:
        return findGeocode(city)
    except:
        return findGeocode(city)

# each value from city column will be fetched and sent to function find_geocode
for i in (df_metropolitan["metropolitanAreas"]):

    if findGeocode(i) != None:

        loc = findGeocode(i)

        # coordinates returned from function is stored into two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    # if coordinate for a city not found, insert "NaN" indicating missing value
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [13]:
# now add each column to dataframe
df_metropolitan["longitude"] = longitude
df_metropolitan["latitude"] = latitude

In [14]:
df_metropolitan

,metropolitanAreas,longitude,latitude
0,Allen,-95.317021,37.888217
1,Warsaw,21.006725,52.231958
2,Kagoshima,130.547408,31.521587
3,Mexicali,-115.448325,32.624862
4,Leicester,-1.133079,52.636140
...,...,...,...
660,Cass,-94.351037,33.056086
661,Carmen,125.674482,7.375394
662,Linn,-91.603793,42.063183
663,Lafayette (IN),-86.890423,40.403467


# Merge and write to csv

In [15]:
# merge the dataframes
df_oecdCoord = pd.merge(oecd_df, df_metropolitan, on="metropolitanAreas")

# write the dataframe to csv
df_oecdCoord.to_csv('Data/CITIES_ALL', header="True", sep="|", doublequote=True)


# Transpose data matrix into row = cities, columns = variables (TIBOR)

In [16]:
#Now we're going to transform the matrix such that on the rows we find countries, whereas on the columns we find variables
#In principle, such matrix could be built for every year

#first get rid of all unnecessecary columns
df_oecd_col_red = df_oecdCoord[['metroId','metropolitanAreas','latitude','longitude','var','variables','year','value']]

#first test some things on 1 year
df_oecd_col_red_14 = df_oecd_col_red[df_oecd_col_red['year']==2014]
#df_oecd_col_red_14.head()

#group by metropolitanAreas
df_total_14 = df_oecd_col_red_14[['metroId','metropolitanAreas','latitude','longitude','var','variables','value']]

df_total_14 = df_total_14.pivot_table(index=['metroId','metropolitanAreas','latitude','longitude'], 
                    columns=['var','variables'], 
                    values='value', 
                    aggfunc='mean')
df_total_14

,,,var,CDD,ELD_DEP_RA,EMP_IND_TOTAL,EMP_RA_15_64,EMP_RA_15_64_GR_2001,EMP_RA_15_64_GR_2007,EMP_RA_15_MAX,EMP_RA_15_MAX_GR_2001,EMP_RA_15_MAX_GR_2007,EMP_Y15_64,...,UNEM_RA_15_MAX,UNEM_Y15_64,UNEM_Y15_64_SH_NAT,UNEM_Y15_MAX,UNEM_Y15_MAX_SH_NAT,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,URB_AREA_PC,YOU_DEP_RA
,,,variables,Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C),Elderly dependency ratio (65+ over population 15-64),Employment at place of work,Employment rate (employment 15-64 over population 15-64),"Employment rate growth index (2001=100), 15-64 years old","Employment rate growth index (2007=100), 15-64 years old",Employment rate (employment 15+ over population 15+),"Employment rate growth index (2001=100), 15 years old and over","Employment rate growth index (2007=100), 15 years old and over",Employment (15-64 years old),...,Unemployment rate (unemployment 15+ over labour force 15+),Unemployment (15-64 years old),Unemployment (15-64 years old) of the metropolitan area as a share of the national value,Unemployment (15 years old and over),Unemployment (15 years old and over) of the metropolitan area as share of the national value,Urbanised area (built-up area or land for urban use in km2),Urbanised area of the core area (km2),Urbanised area of the hinterland area (km2),Urbanised area per capita (m2 per capita),Youth dependency ratio (-15 over population 15-64)
metroId,metropolitanAreas,latitude,longitude,,,,,,,,,,,,,,,,,,,,,
AT001,Vienna,48.208354,16.372504,23.0,26.2,1408986.0,66.1,96.7,101.0,54.4,96.0,96.8,1263282.0,...,7.6,116316.0,47.6,107269.0,43.8,858.0,222.0,636.0,303.0,21.1
AT002,Graz,47.070868,15.438279,2.0,25.8,337576.0,65.4,106.5,92.6,54.3,97.3,93.9,281151.0,...,4.8,15631.0,6.4,14906.0,6.1,248.0,59.0,189.0,393.0,19.8
AT003,Linz,48.305908,14.286198,2.0,25.9,341180.0,71.3,102.7,100.4,60.7,104.9,99.6,300108.0,...,4.2,NaN,NaN,13948.0,5.7,189.0,43.0,146.0,303.0,22.1
AT004,Salzburg,47.798135,13.046481,1.0,26.2,211410.0,73.5,104.0,103.9,58.1,97.0,92.7,178429.0,...,3.6,6654.0,2.7,6615.0,2.7,97.0,28.0,69.0,268.0,21.8
AT005,Innsbruck,47.265430,11.392769,0.0,25.1,189869.0,69.1,103.0,97.5,58.7,103.9,94.8,158927.0,...,3.4,NaN,NaN,5987.0,2.4,108.0,19.0,88.0,322.0,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA95,Ingham,42.586050,-84.380084,15.0,19.5,NaN,NaN,NaN,NaN,58.1,87.9,93.2,NaN,...,5.9,NaN,NaN,14267.0,0.1,271.0,141.0,130.0,578.0,25.0
USA96,Montgomery (AL),33.810124,-86.809849,550.0,20.9,245777.0,NaN,NaN,NaN,43.8,94.3,95.2,NaN,...,6.6,NaN,NaN,11268.0,0.1,246.0,154.0,92.0,528.0,28.6


In [17]:
print(df_total_14.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 667 entries, ('AT001', 'Vienna', 48.2083537, 16.3725042) to ('USA99', 'Mahoning', 40.9325643, -79.4592102)
Data columns (total 75 columns):
 #   Column                                                                                                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                              --------------  -----  
 0   (CDD, Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C))  663 non-null    float64
 1   (ELD_DEP_RA, Elderly dependency ratio (65+ over population 15-64))                

In [18]:
#Now we only select the useful variables: i.e. aggregate variables, relative variables, ...
df_selection_14 = df_total_14[['CDD','URB_AREA','URB_AREA_CORE',
                               'URB_AREA_HINTER','FRAGMENTATION',
                               'GDP_PC_REAL_PPP','PARTIC_RA_15_64','POP_DEN',
                              'POP_TOT_GI','PWM_EX_CORE','T_Y0_14_SH_NAT',
                               'T_Y15_64_SH_NAT','T_Y65_MAX_SH_NAT'
                              ]]
df_selection_14

#NOTE : This is a preliminary reference dataset

,,,var,CDD,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,FRAGMENTATION,GDP_PC_REAL_PPP,PARTIC_RA_15_64,POP_DEN,POP_TOT_GI,PWM_EX_CORE,T_Y0_14_SH_NAT,T_Y15_64_SH_NAT,T_Y65_MAX_SH_NAT
,,,variables,Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C),Urbanised area (built-up area or land for urban use in km2),Urbanised area of the core area (km2),Urbanised area of the hinterland area (km2),Territorial fragmentation (local governments per 100 000 inhabitants),"GDP per capita (USD, constant prices, constant PPP, base year 2015)",Participation rate (labour force 15-64 over population 15-64),Population density (inhabitants per km2),Growth/shrinking index of the total population (2001=100),Mean population exposure to PM2.5 air pollution in the core area,Youth population (0-14) of the metropolitan area as a share of the national youth population,Working age population (15-64) of the metropolitan area as a share of the national working age population,Elderly population (0-14) of the metropolitan area as a share of the national elderly population
metroId,metropolitanAreas,latitude,longitude,,,,,,,,,,,,,
AT001,Vienna,48.208354,16.372504,23.0,858.0,222.0,636.0,12.4,53992.0,72.2,293.0,112.5,14.9,33.1,33.3,32.2
AT002,Graz,47.070868,15.438279,2.0,248.0,59.0,189.0,16.3,50492.0,69.8,202.0,110.8,17.7,7.0,7.5,7.1
AT003,Linz,48.305908,14.286198,2.0,189.0,43.0,146.0,22.6,53792.0,74.8,175.0,105.3,15.3,7.6,7.3,7.0
AT004,Salzburg,47.798135,13.046481,1.0,97.0,28.0,69.0,15.9,62784.0,76.3,232.0,105.5,12.3,4.3,4.2,4.1
AT005,Innsbruck,47.265430,11.392769,0.0,108.0,19.0,88.0,23.0,53602.0,71.7,132.0,110.4,12.6,3.9,4.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA95,Ingham,42.586050,-84.380084,15.0,271.0,141.0,130.0,7.2,42603.0,NaN,106.0,104.3,8.2,0.1,0.2,0.1
USA96,Montgomery (AL),33.810124,-86.809849,550.0,246.0,154.0,92.0,9.3,41655.0,NaN,33.0,105.4,10.6,0.1,0.1,0.1


In [61]:
#Next we add (1) construction ratio and (2) Tree cover manually as variables

#Construction_data
construction_ratio_14 = oecd_data5_improved[oecd_data5_improved['Year_x']==2014]
construction_ratio_14 = construction_ratio_14[['METRO_ID','construction_ratio']]

#Tree_cover
tree_cover = oecd_data6[oecd_data6['VAR']=='TREECOVER_SHARE_CORE']
tree_cover = tree_cover[tree_cover['Year']==2015]
tree_cover = tree_cover[['METRO_ID','Value']]

tree_and_construction = pd.merge(construction_ratio_14,tree_cover,how='inner',on='METRO_ID')
tree_and_construction = tree_and_construction.rename(columns = {'METRO_ID':'metroId','construction_ratio': 'CONSTR_RAT', 'Value': 'TREECOVER_SHARE_CORE'}, inplace = False)

#Merge with main dataset
df_extended_14 = pd.merge(df_selection_14,tree_and_construction,how='outer',on='metroId')
df_extended_14

#NOTE: SOMEHOW, now the 'metropolitanAreas' have disappeared. To be looked into later



c:\users\geoff\anaconda3\envs\moderndata1\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,metroId,"(CDD, Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C))","(URB_AREA, Urbanised area (built-up area or land for urban use in km2))","(URB_AREA_CORE, Urbanised area of the core area (km2))","(URB_AREA_HINTER, Urbanised area of the hinterland area (km2))","(FRAGMENTATION, Territorial fragmentation (local governments per 100 000 inhabitants))","(GDP_PC_REAL_PPP, GDP per capita (USD, constant prices, constant PPP, base year 2015))","(PARTIC_RA_15_64, Participation rate (labour force 15-64 over population 15-64))","(POP_DEN, Population density (inhabitants per km2))","(POP_TOT_GI, Growth/shrinking index of the total population (2001=100))","(PWM_EX_CORE, Mean population exposure to PM2.5 air pollution in the core area)","(T_Y0_14_SH_NAT, Youth population (0-14) of the metropolitan area as a share of the national youth population)","(T_Y15_64_SH_NAT, Working age population (15-64) of the metropolitan area as a share of the national working age population)","(T_Y65_MAX_SH_NAT, Elderly population (0-14) of the metropolitan area as a share of the national elderly population)",CONSTR_RAT,TREECOVER_SHARE_CORE
0,AT001,23.0,858.0,222.0,636.0,12.4,53992.0,72.2,293.0,112.5,14.9,33.1,33.3,32.2,0.089217,22.1
1,AT002,2.0,248.0,59.0,189.0,16.3,50492.0,69.8,202.0,110.8,17.7,7.0,7.5,7.1,0.080155,32.9
2,AT003,2.0,189.0,43.0,146.0,22.6,53792.0,74.8,175.0,105.3,15.3,7.6,7.3,7.0,0.053209,15.6
3,AT004,1.0,97.0,28.0,69.0,15.9,62784.0,76.3,232.0,105.5,12.3,4.3,4.2,4.1,0.062743,18.8
4,AT005,0.0,108.0,19.0,88.0,23.0,53602.0,71.7,132.0,110.4,12.6,3.9,4.0,3.7,0.042671,56.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,USA96,550.0,246.0,154.0,92.0,9.3,41655.0,NaN,33.0,105.4,10.6,0.1,0.1,0.1,0.017839,48.7
664,USA97,141.0,253.0,242.0,11.0,1.3,49942.0,NaN,233.0,115.5,9.1,0.1,0.1,0.1,0.126122,27.8
665,USA98,221.0,251.0,251.0,NaN,4.4,53205.0,NaN,233.0,124.8,8.9,0.2,0.2,0.1,0.121667,46.6
666,USA99,23.0,290.0,151.0,139.0,9.6,38427.0,NaN,159.0,91.5,9.5,0.1,0.1,0.2,0.105340,20.8


# Data Cleaning

In [20]:
# Start from here

In [62]:
df_extended_14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 0 to 667
Data columns (total 16 columns):
 #   Column                                                                                                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                              --------------  -----  
 0   metroId                                                                                                                                                                                                             668 non-null    object 
 1   (CDD, Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the dail

In [63]:
df_extended_14.describe()

,"(CDD, Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C))","(URB_AREA, Urbanised area (built-up area or land for urban use in km2))","(URB_AREA_CORE, Urbanised area of the core area (km2))","(URB_AREA_HINTER, Urbanised area of the hinterland area (km2))","(FRAGMENTATION, Territorial fragmentation (local governments per 100 000 inhabitants))","(GDP_PC_REAL_PPP, GDP per capita (USD, constant prices, constant PPP, base year 2015))","(PARTIC_RA_15_64, Participation rate (labour force 15-64 over population 15-64))","(POP_DEN, Population density (inhabitants per km2))","(POP_TOT_GI, Growth/shrinking index of the total population (2001=100))","(PWM_EX_CORE, Mean population exposure to PM2.5 air pollution in the core area)","(T_Y0_14_SH_NAT, Youth population (0-14) of the metropolitan area as a share of the national youth population)","(T_Y15_64_SH_NAT, Working age population (15-64) of the metropolitan area as a share of the national working age population)","(T_Y65_MAX_SH_NAT, Elderly population (0-14) of the metropolitan area as a share of the national elderly population)",CONSTR_RAT,TREECOVER_SHARE_CORE
count,663.000000,667.000000,667.000000,530.000000,634.000000,489.000000,235.000000,667.000000,516.000000,666.000000,630.000000,630.000000,630.000000,668.000000,668.000000
mean,188.291101,325.499250,221.775112,130.505660,6.975710,44307.977505,73.219574,458.406297,109.734496,13.777027,2.796667,2.829048,2.642698,0.105221,25.636677
std,362.390879,620.411322,521.515193,178.071822,11.517911,14896.912757,6.604178,541.853679,12.508747,6.256057,7.173308,7.247996,6.776031,0.094589,22.287604
min,0.000000,2.000000,2.000000,0.000000,0.100000,7580.000000,48.000000,9.000000,81.200000,4.500000,0.100000,0.100000,0.100000,0.000126,0.000000
25%,5.000000,85.000000,38.000000,36.000000,0.800000,35018.000000,71.000000,143.500000,101.475000,9.300000,0.300000,0.300000,0.300000,0.040028,6.675000
50%,28.000000,147.000000,79.000000,74.000000,3.400000,42329.000000,74.300000,287.000000,107.950000,12.200000,0.700000,0.700000,0.700000,0.076817,19.450000
75%,168.000000,290.000000,187.000000,139.750000,7.500000,53602.000000,77.250000,558.500000,115.750000,15.975000,1.800000,1.875000,1.800000,0.141414,39.575000
max,2391.000000,6718.000000,6718.000000,1358.000000,107.800000,121011.000000,92.700000,4304.000000,185.900000,36.400000,99.700000,100.400000,100.700000,0.693385,92.200000


In [64]:
df_extended_14.columns

Index([                                                                                                                                                                                                             'metroId',
       ('CDD', 'Annual Cooling Degree Days (Sum over a year of the differences between the daily mean outdoor air temperature and the threshold temperature (22Â°C) when the daily mean outdoor temperature is above 22Â°C)'),
                                                                                                                                                  ('URB_AREA', 'Urbanised area (built-up area or land for urban use in km2)'),
                                                                                                                                                                   ('URB_AREA_CORE', 'Urbanised area of the core area (km2)'),
                                                                                                            

In [65]:
df_extended_14.columns = ['metroId','CDD','URB_AREA','URB_AREA_CORE','URB_AREA_HINTER','FRAGMENTATION','GDP_PC_REAL_PPP','PARTIC_RA_15_64','POP_DEN','POP_TOT_GI','PWM_EX_CORE','T_Y0_14_SH_NAT','T_Y15_64_SH_NAT','T_Y65_MAX_SH_NAT','CONSTR_RAT','TREECOVER_SHARE_CORE']

In [66]:
df_extended_14.isnull().sum()

metroId                   0
CDD                       5
URB_AREA                  1
URB_AREA_CORE             1
URB_AREA_HINTER         138
FRAGMENTATION            34
GDP_PC_REAL_PPP         179
PARTIC_RA_15_64         433
POP_DEN                   1
POP_TOT_GI              152
PWM_EX_CORE               2
T_Y0_14_SH_NAT           38
T_Y15_64_SH_NAT          38
T_Y65_MAX_SH_NAT         38
CONSTR_RAT                0
TREECOVER_SHARE_CORE      0
dtype: int64

In [67]:
df_extended_14.dropna(subset=['CDD'],inplace=True)
# drop values that have missing data for cooling days

In [58]:
df_extended_14

,metroId,CDD,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,FRAGMENTATION,GDP_PC_REAL_PPP,POP_DEN,POP_TOT_GI,PWM_EX_CORE,T_Y0_14_SH_NAT,T_Y15_64_SH_NAT,T_Y65_MAX_SH_NAT,CONSTR_RAT,TREECOVER_SHARE_CORE
0,AT001,23.0,858.0,222.0,636.0,12.4,53992.0,293.0,112.5,14.9,33.1,33.3,32.2,0.089217,22.1
1,AT002,2.0,248.0,59.0,189.0,16.3,50492.0,202.0,110.8,17.7,7.0,7.5,7.1,0.080155,32.9
2,AT003,2.0,189.0,43.0,146.0,22.6,53792.0,175.0,105.3,15.3,7.6,7.3,7.0,0.053209,15.6
3,AT004,1.0,97.0,28.0,69.0,15.9,62784.0,232.0,105.5,12.3,4.3,4.2,4.1,0.062743,18.8
4,AT005,0.0,108.0,19.0,88.0,23.0,53602.0,132.0,110.4,12.6,3.9,4.0,3.7,0.042671,56.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,USA95,15.0,271.0,141.0,130.0,7.2,42603.0,106.0,104.3,8.2,0.1,0.2,0.1,0.060995,10.6
663,USA96,550.0,246.0,154.0,92.0,9.3,41655.0,33.0,105.4,10.6,0.1,0.1,0.1,0.017839,48.7
664,USA97,141.0,253.0,242.0,11.0,1.3,49942.0,233.0,115.5,9.1,0.1,0.1,0.1,0.126122,27.8
665,USA98,221.0,251.0,251.0,NaN,4.4,53205.0,233.0,124.8,8.9,0.2,0.2,0.1,0.121667,46.6


In [69]:
df_extended_14.drop(['PARTIC_RA_15_64'], axis = 1,inplace=True)
# Remove column because has over 433 NA
df_extended_14.columns



Index(['metroId', 'CDD', 'URB_AREA', 'URB_AREA_CORE', 'URB_AREA_HINTER',
       'FRAGMENTATION', 'GDP_PC_REAL_PPP', 'POP_DEN', 'POP_TOT_GI',
       'PWM_EX_CORE', 'T_Y0_14_SH_NAT', 'T_Y15_64_SH_NAT', 'T_Y65_MAX_SH_NAT',
       'CONSTR_RAT', 'TREECOVER_SHARE_CORE'],
      dtype='object')

In [70]:
df_extended_14

,metroId,CDD,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,FRAGMENTATION,GDP_PC_REAL_PPP,POP_DEN,POP_TOT_GI,PWM_EX_CORE,T_Y0_14_SH_NAT,T_Y15_64_SH_NAT,T_Y65_MAX_SH_NAT,CONSTR_RAT,TREECOVER_SHARE_CORE
0,AT001,23.0,858.0,222.0,636.0,12.4,53992.0,293.0,112.5,14.9,33.1,33.3,32.2,0.089217,22.1
1,AT002,2.0,248.0,59.0,189.0,16.3,50492.0,202.0,110.8,17.7,7.0,7.5,7.1,0.080155,32.9
2,AT003,2.0,189.0,43.0,146.0,22.6,53792.0,175.0,105.3,15.3,7.6,7.3,7.0,0.053209,15.6
3,AT004,1.0,97.0,28.0,69.0,15.9,62784.0,232.0,105.5,12.3,4.3,4.2,4.1,0.062743,18.8
4,AT005,0.0,108.0,19.0,88.0,23.0,53602.0,132.0,110.4,12.6,3.9,4.0,3.7,0.042671,56.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,USA95,15.0,271.0,141.0,130.0,7.2,42603.0,106.0,104.3,8.2,0.1,0.2,0.1,0.060995,10.6
663,USA96,550.0,246.0,154.0,92.0,9.3,41655.0,33.0,105.4,10.6,0.1,0.1,0.1,0.017839,48.7
664,USA97,141.0,253.0,242.0,11.0,1.3,49942.0,233.0,115.5,9.1,0.1,0.1,0.1,0.126122,27.8
665,USA98,221.0,251.0,251.0,NaN,4.4,53205.0,233.0,124.8,8.9,0.2,0.2,0.1,0.121667,46.6


In [35]:
df_extended_14[df_extended_14['CDD'] == df_extended_14['CDD'].max() ]

,metroId,CDD,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,FRAGMENTATION,GDP_PC_REAL_PPP,PARTIC_RA_15_64,POP_DEN,POP_TOT_GI,PWM_EX_CORE,T_Y0_14_SH_NAT,T_Y15_64_SH_NAT,T_Y65_MAX_SH_NAT,CONSTR_RAT,TREECOVER_SHARE_CORE
56,COL04,2391.0,155.0,117.0,37.0,0.6,11429.0,69.0,1586.0,119.7,19.1,4.5,4.7,4.2,0.111913,5.0


In [36]:
df_extended_14[df_extended_14['CDD'] >366 ]

,metroId,CDD,URB_AREA,URB_AREA_CORE,URB_AREA_HINTER,FRAGMENTATION,GDP_PC_REAL_PPP,PARTIC_RA_15_64,POP_DEN,POP_TOT_GI,PWM_EX_CORE,T_Y0_14_SH_NAT,T_Y15_64_SH_NAT,T_Y65_MAX_SH_NAT,CONSTR_RAT,TREECOVER_SHARE_CORE
56,COL04,2391.0,155.0,117.0,37.0,0.6,11429.0,69.0,1586.0,119.7,19.1,4.5,4.7,4.2,0.111913,5.0
57,COL05,2176.0,97.0,76.0,21.0,0.6,10104.0,59.6,607.0,116.5,18.4,2.5,2.5,2.4,0.049039,21.8
58,COL06,465.0,59.0,59.0,NaN,0.4,21455.0,76.5,905.0,113.7,26.8,2.0,2.5,2.6,0.047929,54.0
59,COL07,1179.0,93.0,93.0,NaN,0.5,7580.0,68.5,540.0,117.1,29.9,1.7,1.7,1.6,0.061876,39.5
63,COL11,369.0,41.0,41.0,NaN,0.2,NaN,NaN,202.0,121.4,19.0,1.1,1.0,0.8,0.017358,82.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,USA75,426.0,329.0,285.0,44.0,7.9,47523.0,NaN,63.0,110.3,8.1,0.2,0.2,0.2,0.034833,0.1
648,USA81,881.0,542.0,542.0,NaN,4.9,37531.0,NaN,165.0,114.2,6.4,0.1,0.2,0.3,0.160974,33.2
653,USA86,435.0,200.0,200.0,NaN,5.7,40316.0,NaN,134.0,113.9,11.0,0.2,0.2,0.1,0.050904,7.9
657,USA90,806.0,465.0,465.0,NaN,4.7,30173.0,NaN,154.0,112.3,6.7,0.1,0.1,0.3,0.141337,62.3


In [40]:
df_extended_14.dtypes

metroId                  object
CDD                     float64
URB_AREA                float64
URB_AREA_CORE           float64
URB_AREA_HINTER         float64
FRAGMENTATION           float64
GDP_PC_REAL_PPP         float64
PARTIC_RA_15_64         float64
POP_DEN                 float64
POP_TOT_GI              float64
PWM_EX_CORE             float64
T_Y0_14_SH_NAT          float64
T_Y15_64_SH_NAT         float64
T_Y65_MAX_SH_NAT        float64
CONSTR_RAT              float64
TREECOVER_SHARE_CORE    float64
dtype: object